|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 5:</h2>|<h1>Observation (non-causal) mech interp<h1>|
|<h2>Section:</h2>|<h1>Investigating token embeddings<h1>|
|<h2>Lecture:</h2>|<h1><b>CodeChallenge: Cosine similarity (advanced)<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import torch

# high res matplotlib
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [ ]:
from transformers import GPT2Model,GPT2Tokenizer

# import GPT-2 model, extract tokenizer and embeddings
gpt2 = GPT2Model.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
embedding = gpt2.wte.weight.detach().numpy()

# Exercise 1: Manual and Pytorch cosine similarity

In [ ]:
# pick two random tokens
tokenpair = np.random.choice(np.arange(3000,6001),2)

# get their embedding vectors
v1 = embedding[tokenpair[0]]
v2 = embedding[tokenpair[1]]

print(f'Token pair: "{tokenizer.decode(tokenpair[0])}" and "{tokenizer.decode(tokenpair[1])}"')
print(f'Embedding shape: {v1.shape}')

In [ ]:
# calculate cosine similarity manually
num = np.sum(v1*v2)
norm_v1 = np.sum(v1**2)
norm_v2 = np.sum(v2**2)
den = np.sqrt( norm_v1*norm_v2 )

print(f'Shape of vectors: {v1.shape}')
manual_cs = num/den

In [ ]:
# and now in torch
v1_torch = torch.tensor(v1).unsqueeze(dim=0)
v2_torch = torch.tensor(v2).view(1,-1) # both view() and unsqueeze() work in this case

print(f'Shape of torch vectors: {v1_torch.shape}')
torch_cs = torch.cosine_similarity(v1_torch,v2_torch)

In [ ]:
# print the results
print(f'Manual cosine similarity:  {manual_cs:.5f}')
print(f'Pytorch cosine similarity: {torch_cs.item():.5f}')

# Exercise 2: Matrix of pairwise similarities

In [ ]:
# using a for-loop
numtoks = 30

E = embedding[:numtoks,:]

# FYI, mean-centering would give correlation coefficient
# E -= np.mean(E,axis=1,keepdims=True)

print(f'Embedding submatrix has size {E.shape}')

cs_matrix = np.zeros((numtoks,numtoks))

for i in range(numtoks):
  for j in range(numtoks):

    # cosine similarity
    num = np.sum(E[i,:]*E[j,:])
    den = np.sqrt( np.sum(E[i,:]**2)*np.sum(E[j,:]**2) )

    # slot into the matrix
    cs_matrix[i,j] = num/den


# and show the matrix
plt.figure(figsize=(6,5))

plt.imshow(cs_matrix,vmin=.2,vmax=.8)
ticklabels = [tokenizer.decode(t) for t in np.arange(numtoks)]
plt.gca().set(xticks=range(numtoks),xticklabels=ticklabels,
              yticks=range(numtoks),yticklabels=ticklabels)
plt.xticks(rotation=45)
plt.colorbar(pad=.02)

plt.show()

In [ ]:
# repeat using matrix multiplication in pytorch
Et = torch.tensor(E) # first convert to pytorch tensor

# normalize (note the vector_norm not matrix_norm!)
Et_norm = Et / torch.linalg.vector_norm(Et,axis=1,keepdims=True)

# cosine similarity matrix
cs_matrixT = Et_norm @ Et_norm.T

# need to conver to numpy first :P
print(f'Mean absolute difference = {np.mean(abs(cs_matrixT.numpy()-cs_matrix)):.10f}')

# Exercise 3: Softmaxify the cosine similarities matrix

In [ ]:
numtoks = 6

Et = torch.tensor( embedding[:numtoks,:] )
Et_norm = Et / torch.linalg.vector_norm(Et,axis=1,keepdims=True)
cs_matrixP = Et_norm @ Et_norm.T
cs_matrixP

In [ ]:
# create a mask of -infs
infmask = torch.tril(torch.full((numtoks,numtoks),-np.inf))

# mask the cosine similarities
mat4softmax = cs_matrixP + infmask

print(infmask), print('')
print(mat4softmax)

In [ ]:
# softmax!
softmat = torch.exp(mat4softmax) / torch.sum(torch.exp(mat4softmax))

# print the results
print(softmat)
print(f'\nSum over all matrix elements = {torch.sum(softmat)}')

In [ ]:
# nan mask
nanmask = torch.tril(torch.full((numtoks,numtoks),torch.nan))
nanmask

In [ ]:
# visualize the relationship between softmax(cs) and cs
C = cs_matrixP.clone()
S = softmat.clone()

# replace lower-triangle with nan's
C += nanmask
S += nanmask

print(S)

# and plot
plt.plot(C.flatten(),S.flatten(),'ko')
plt.gca().set(xlabel='Cosine similarity',ylabel='Softmax-cosine')
plt.show()

# Exercise 4: One-to-all similarity

In [ ]:
randtoken = np.random.randint(0,tokenizer.vocab_size,1)

# vector,matrix
one2all_cs = torch.cosine_similarity(torch.tensor(embedding[randtoken,:]),torch.tensor(embedding))
print(f'Size of embedding matrix: {embedding.shape}')
print(f'Size of one2all_cs: {one2all_cs.shape}')

In [ ]:
# setup the figure
fig = plt.figure(figsize=(12,4))
gs = GridSpec(1,4)
ax0 = fig.add_subplot(gs[:-1])
ax1 = fig.add_subplot(gs[-1])

# plot all similarities
ax0.plot(one2all_cs,'ko',markerfacecolor='gray',alpha=.5)
ax0.set(xlabel='Token index',ylabel='Cosine similarity',xlim=[-10,len(one2all_cs)+10],title=f'Cosine similarity of "{tokenizer.decode(randtoken)}" with all tokens')

# their distribution
ax1.hist(one2all_cs,bins=40,color='k',alpha=.5)
ax1.set(xlabel='Cosine similarity',ylabel='Count',yticks=[],title='Distribution of cs')

plt.tight_layout()
plt.show()

In [ ]:
# torch.topk to find the 10 closest embeddings
values,indices = torch.topk(one2all_cs,10)

print(f'Top 10 similar tokens to "{tokenizer.decode(randtoken)}":')
for v,i in zip(values,indices):
  print(f'  Sc = {v:.3f} with token "{tokenizer.decode(i)}"')